# Experiment 001: Baseline Evaluation

Evaluate the pre-optimized santa-2025.csv solution and calculate its score.

In [1]:
import pandas as pd
import numpy as np
import math
from decimal import Decimal, getcontext

# Set high precision for calculations
getcontext().prec = 30

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
NV = 15

In [2]:
def parse_value(s):
    """Parse a value that may be prefixed with 's'"""
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def get_poly(cx, cy, deg):
    """Get polygon vertices for a tree at position (cx, cy) with rotation deg"""
    rad = math.radians(deg)
    s, c = math.sin(rad), math.cos(rad)
    px, py = [], []
    for i in range(NV):
        x = TX[i] * c - TY[i] * s + cx
        y = TX[i] * s + TY[i] * c + cy
        px.append(x)
        py.append(y)
    return px, py

def get_bounding_box(px, py):
    """Get bounding box of a polygon"""
    return min(px), max(px), min(py), max(py)

def calculate_side_for_n(df, n):
    """Calculate the bounding box side length for n trees"""
    # Filter rows for this n
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    
    if len(rows) == 0:
        return None
    
    all_x, all_y = [], []
    for _, row in rows.iterrows():
        cx = parse_value(row['x'])
        cy = parse_value(row['y'])
        deg = parse_value(row['deg'])
        px, py = get_poly(cx, cy, deg)
        all_x.extend(px)
        all_y.extend(py)
    
    minx, maxx = min(all_x), max(all_x)
    miny, maxy = min(all_y), max(all_y)
    
    return max(maxx - minx, maxy - miny)

def calculate_total_score(df):
    """Calculate total score = sum(side^2 / n) for n=1 to 200"""
    total = 0.0
    sides = []
    for n in range(1, 201):
        side = calculate_side_for_n(df, n)
        if side is not None:
            score_n = (side ** 2) / n
            total += score_n
            sides.append((n, side, score_n))
    return total, sides

In [3]:
# Load the pre-optimized solution
preopt_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
df = pd.read_csv(preopt_path)
print(f"Loaded {len(df)} rows")
print(df.head(10))

Loaded 20100 rows
      id                       x                       y  \
0  001_0    s-48.196086194214246     s58.770984615214225   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   
5  003_2      s0.641714640229075      s1.180458566613381   
6  004_0  s-0.324747789589372171   s0.132109978088185392   
7  004_1   s0.315354346242637695   s0.132109978063475492   
8  004_2   s0.324747789592379210  s-0.732109978069475531   
9  004_3  s-0.315354348134818330  s-0.732109978094185987   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  
5      s155.13405193710082  
6  s156.370622145636389178  
7  s156.370622269264089255  
8  s336.370622269264003990  
9  s336.370622145636446021  


In [4]:
# Calculate the total score
total_score, sides = calculate_total_score(df)
print(f"\nTotal Score: {total_score:.6f}")
print(f"Target Score: 68.919154")
print(f"Gap: {total_score - 68.919154:.6f} ({(total_score - 68.919154) / 68.919154 * 100:.2f}%)")


Total Score: 70.676102
Target Score: 68.919154
Gap: 1.756948 (2.55%)


In [5]:
# Show the top 20 contributors to the score (highest score contribution)
sides_sorted = sorted(sides, key=lambda x: x[2], reverse=True)
print("\nTop 20 score contributors (n, side, score_contribution):")
for n, side, score_n in sides_sorted[:20]:
    print(f"  N={n:3d}: side={side:.6f}, contribution={score_n:.6f}")


Top 20 score contributors (n, side, score_contribution):
  N=  1: side=0.813173, contribution=0.661250
  N=  2: side=0.949504, contribution=0.450779
  N=  3: side=1.142031, contribution=0.434745
  N=  5: side=1.443692, contribution=0.416850
  N=  4: side=1.290806, contribution=0.416545
  N=  7: side=1.673104, contribution=0.399897
  N=  6: side=1.548438, contribution=0.399610
  N=  9: side=1.867280, contribution=0.387415
  N=  8: side=1.755921, contribution=0.385407
  N= 15: side=2.384962, contribution=0.379203
  N= 10: side=1.940696, contribution=0.376630
  N= 21: side=2.811667, contribution=0.376451
  N= 20: side=2.742469, contribution=0.376057
  N= 11: side=2.033002, contribution=0.375736
  N= 22: side=2.873270, contribution=0.375258
  N= 16: side=2.446640, contribution=0.374128
  N= 26: side=3.118320, contribution=0.373997
  N= 12: side=2.114873, contribution=0.372724
  N= 13: side=2.200046, contribution=0.372323
  N= 25: side=3.050182, contribution=0.372144


In [6]:
# Show small N values (1-10) - these have highest impact
print("\nSmall N values (1-10):")
for n, side, score_n in sides[:10]:
    print(f"  N={n:3d}: side={side:.6f}, contribution={score_n:.6f}")


Small N values (1-10):
  N=  1: side=0.813173, contribution=0.661250
  N=  2: side=0.949504, contribution=0.450779
  N=  3: side=1.142031, contribution=0.434745
  N=  4: side=1.290806, contribution=0.416545
  N=  5: side=1.443692, contribution=0.416850
  N=  6: side=1.548438, contribution=0.399610
  N=  7: side=1.673104, contribution=0.399897
  N=  8: side=1.755921, contribution=0.385407
  N=  9: side=1.867280, contribution=0.387415
  N= 10: side=1.940696, contribution=0.376630


In [7]:
# Copy the pre-optimized solution as our baseline submission
import shutil
shutil.copy(preopt_path, '/home/submission/submission.csv')
print("Copied pre-optimized solution to /home/submission/submission.csv")

Copied pre-optimized solution to /home/submission/submission.csv


In [8]:
# Verify the submission format
submission = pd.read_csv('/home/submission/submission.csv')
print(f"Submission shape: {submission.shape}")
print(f"Columns: {submission.columns.tolist()}")
print(submission.head())

Submission shape: (20100, 4)
Columns: ['id', 'x', 'y', 'deg']
      id                       x                       y  \
0  001_0    s-48.196086194214246     s58.770984615214225   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  
